In [ ]:
#%pip install -r requirements.txt



In [1]:
import duckdb
cn = duckdb.connect("c2_workouts.db")
cn.sql("create or replace view v_detail_workouts as select * from read_csv_auto('~/concept2/workouts/*detail*.csv')")
cn.sql("create or replace view v_summary_workouts as select * from read_csv_auto('~/concept2/workouts/*summary*.csv')")   

# MotherDuck MCP Server Integration

This notebook demonstrates how to use the MotherDuck MCP server to run natural language queries against your DuckDB database.

The MCP server is running and listening for connections. You can now use natural language to query your data!

In [2]:
# Test the connection to see what tables are available
result = cn.sql("SHOW TABLES").fetchall()
print("Available tables:")
for table in result:
    print(f"- {table[0]}")

# Test a simple query
result = cn.sql("SELECT COUNT(*) as total_rows FROM v_summary_workouts").fetchall()
print(f"\nTotal rows in summary workouts: {result[0][0]}")

Available tables:
- v_detail_workouts
- v_summary_workouts

Total rows in summary workouts: 221


In [ ]:
# Install MCP client for making natural language queries
# %pip install mcp

# For now, let's demonstrate direct SQL queries that you could translate from natural language
# Example natural language query: "Show me the average workout duration by month"

import pandas as pd

# Let's first see what columns are available in our views
result = cn.sql("DESCRIBE v_summary_workouts").fetchall()
print("Columns in v_summary_workouts:")
for col in result:
    print(f"- {col[0]} ({col[1]})")
    
print("\n" + "="*50 + "\n")

# Sample the data to see what we're working with
result = cn.sql("SELECT * FROM v_summary_workouts LIMIT 5").fetchall()
df = pd.DataFrame(result)
print("Sample data from v_summary_workouts:")
print(df)

In [ ]:
# Close the direct database connection to allow MCP server to connect
cn.close()

# Install and use the MCP client for true natural language queries
import subprocess
import json
import asyncio

# Install required packages
try:
    import nest_asyncio
except ImportError:
    subprocess.check_call(["pip", "install", "nest_asyncio"])
    import nest_asyncio

# MCP Client setup for natural language queries
class MCPDuckDBClient:
    def __init__(self):
        self.server_process = None
        
    async def start_server(self):
        # Start the MCP server
        self.server_process = await asyncio.create_subprocess_exec(
            "uv", "run", "mcp-server-motherduck", "--db-path", "./c2_workouts.db",
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
            stdin=asyncio.subprocess.PIPE
        )
        
        # Wait for server to start
        await asyncio.sleep(3)
        
    async def query(self, natural_language_question):
        """Send a natural language question to the MCP server"""
        
        # First, let's provide context about the database schema
        schema_context = """
        Database contains workout data with these views:
        - v_summary_workouts: Contains workout summaries with columns like Date, Time, Distance
        - v_detail_workouts: Contains detailed workout data
        
        The data comes from Concept2 rowing machine CSV exports.
        """
        
        # Create the query with context
        full_query = f"{schema_context}\n\nUser question: {natural_language_question}"
        
        request = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "tools/call",
            "params": {
                "name": "execute_query",
                "arguments": {
                    "sql": f"-- {full_query}\nSELECT 'Processing your question about the workout data...' as status"
                }
            }
        }
        
        try:
            # Send request to MCP server
            request_json = json.dumps(request) + "\n"
            self.server_process.stdin.write(request_json.encode())
            await self.server_process.stdin.drain()
            
            # Read response
            response_line = await self.server_process.stdout.readline()
            if response_line:
                response = json.loads(response_line.decode())
                return response.get("result", {})
            else:
                return {"error": "No response from server"}
                
        except Exception as e:
            return {"error": f"Query failed: {str(e)}"}
    
    def stop_server(self):
        if self.server_process:
            self.server_process.terminate()

# Create MCP client
mcp_client = MCPDuckDBClient()

print("🚀 Setting up MCP client for natural language queries...")
print("📊 Database context: Concept2 workout data with summary and detail views")
print("Now you can ask questions in plain English!")

🚀 Setting up MCP client for natural language queries...
Now you can ask questions in plain English!


In [4]:
# Now let's ask natural language questions!
async def ask_question(question):
    print(f"❓ Question: {question}")
    try:
        await mcp_client.start_server()
        await asyncio.sleep(2)  # Give server time to start
        
        result = await mcp_client.query(question)
        print(f"🤖 Answer: {result}")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None
    finally:
        mcp_client.stop_server()

# Example natural language queries - no SQL needed!
questions = [
    "How many workouts do I have in total?",
    "What is the average duration of my workouts?", 
    "Show me my 5 most recent workouts",
    "What's the longest workout I've done?",
    "How many workouts did I do last month?"
]

print("🎯 Testing Natural Language Queries with MCP:")
print("=" * 60)

# Run the queries
for question in questions:
    print(f"\n🔍 Asking: '{question}'")
    # Note: In Jupyter, you'll need to run this with await
    # For now, let's prepare the async setup
    print("   (Run the cell below to execute this query)")
    break  # Just show the first example for now

🎯 Testing Natural Language Queries with MCP:

🔍 Asking: 'How many workouts do I have in total?'
   (Run the cell below to execute this query)


In [ ]:
# Test with a direct approach using MCP server capabilities
import nest_asyncio
nest_asyncio.apply()

# Let's test the MCP server with a simple direct query first
async def test_mcp_connection():
    print("🔌 Starting MCP server...")
    await mcp_client.start_server()
    
    # Test basic functionality with SQL
    try:
        # Use MCP server to execute SQL directly
        request = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "tools/call", 
            "params": {
                "name": "execute_query",
                "arguments": {
                    "sql": "SELECT COUNT(*) as total_workouts FROM v_summary_workouts"
                }
            }
        }
        
        # Send to MCP server
        request_json = json.dumps(request) + "\n"
        mcp_client.server_process.stdin.write(request_json.encode())
        await mcp_client.server_process.stdin.drain()
        
        # Read response
        response_line = await mcp_client.server_process.stdout.readline()
        if response_line:
            response = json.loads(response_line.decode()) 
            result = response.get("result", {})
            print(f"✅ MCP Server Response: {result}")
            return result
        else:
            print("❌ No response from MCP server")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None
    finally:
        mcp_client.stop_server()

# Test the MCP connection
result = await test_mcp_connection()
print(f"\n🎯 Total workouts found: {result}")

❓ Question: How many workouts do I have in total?
🤖 Answer: No result
✅ Final result: No result
🤖 Answer: No result
✅ Final result: No result
